In [3]:
#오늘 날짜
import datetime as dt
now=dt.datetime.now()
today = now.strftime('%Y-%m-%d')
today

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

writer = pd.ExcelWriter(today+'us,kor-popu,watch,gen.xlsx')

In [4]:
#미국 요일별 인기순     
us_week_popu_title, us_week_popu_week, us_week_popu_code =[],[],[]

raw = requests.get('https://www.webtoons.com/en/dailySchedule')
html = BeautifulSoup(raw.text,'html.parser')

for i in range(0,320):    
    us_week_popu_title.append(html.select('p.subj')[i].text)   #제목
    if i in range(0,41) : us_week_popu_week.append('mon')   #요일
    elif i in range(41,90) : us_week_popu_week.append('thu')
    elif i in range(90,139) : us_week_popu_week.append('wed')
    elif i in range(139,183) : us_week_popu_week.append('thr')
    elif i in range(183,232) : us_week_popu_week.append('fri')
    elif i in range(232,278) : us_week_popu_week.append('sat')
    elif i in range(278,320) : us_week_popu_week.append('sun')        
    us_week_popu_code.append(re.sub(r'[^0-9]', '', str(html.select('a.daily_card_item')[i].get('href'))[-6:]))  #코드

us_week_popu_df = pd.DataFrame({'제목' : us_week_popu_title,
                        '요일': us_week_popu_week,
                       '코드': us_week_popu_code})

us_week_popu_df.to_excel(writer, sheet_name='us_week_popu_df')

In [5]:
#미국 장르별 인기순     
us_gen_popu_title, us_gen_popu_genre, us_gen_popu_code =[],[],[]
               
raw = requests.get('https://www.webtoons.com/en/drama?sortOrder=READ_COUNT')
html = BeautifulSoup(raw.text,'html.parser')

#전체 웹툰갯수
num_all_en_webtoon = 0
for k in range(0,16):
    num_all_en_webtoon+=int((len(html.select('ul.card_lst')[k])-1)/2)
num_all_en_webtoon


#장르별 갯수
stack = 0
count=['n0','n1','n2','n3','n4','n5','n6','n7','n8','n9','n10','n11','n12','n13','n14','n15']
for c in range(0,16):
    stack = stack + int((len(html.select('ul.card_lst')[c])-1)/2)
    count[c] = stack

#장르
for i in range(0,num_all_en_webtoon):    
    if i<count[0]: us_gen_popu_genre.append('drama')
    elif i<count[1]: us_gen_popu_genre.append('fantasy')
    elif i<count[2]: us_gen_popu_genre.append('comedy')
    elif i<count[3]: us_gen_popu_genre.append('action')
    elif i<count[4]: us_gen_popu_genre.append('slice of life')
    elif i<count[5]: us_gen_popu_genre.append('romance')
    elif i<count[6]: us_gen_popu_genre.append('superhero')
    elif i<count[7]: us_gen_popu_genre.append('sci-fi')
    elif i<count[8]: us_gen_popu_genre.append('thriller')
    elif i<count[9]: us_gen_popu_genre.append('supernatural')
    elif i<count[10]: us_gen_popu_genre.append('mystery')
    elif i<count[11]: us_gen_popu_genre.append('sports')
    elif i<count[12]: us_gen_popu_genre.append('historical')
    elif i<count[13]: us_gen_popu_genre.append('heartwarming')
    elif i<count[14]: us_gen_popu_genre.append('horror')
    elif i<count[15]: us_gen_popu_genre.append('informative')
    
for i in range(0,num_all_en_webtoon):    
    us_gen_popu_title.append(html.select('p.subj')[i].text)   #제목
    us_gen_popu_code.append(re.sub(r'[^0-9]', '', str(html.select('a.card_item')[i].get('href'))[-6:]))  #코드


us_gen_popu_df = pd.DataFrame({'제목' : us_gen_popu_title,
                        '장르': us_gen_popu_genre,
                       '코드': us_gen_popu_code})

us_gen_popu_df.to_excel(writer, sheet_name='us_gen_popu_df')

In [6]:
#한국 요일별 인기순
kor_week_popu_title, kor_week_popu_week, kor_week_popu_code =[],[],[]

for i in ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']:
    raw = requests.get('https://comic.naver.com/webtoon/weekdayList?week='+i+'&order=User&view=image')
    html = BeautifulSoup(raw.text,'html.parser')
    
    #제목
    raw_title_list = html.select('dt')
    for j in raw_title_list[3:] : 
        kor_week_popu_title.append(j.get_text())
    
    container = html.select('ul.img_list li')
    for cont in container:
        #코드
        kor_week_popu_code.append(re.sub(r'[^0-9]', '', str(cont.select_one('dt'))[:45]))
        #요일
        kor_week_popu_week.append(i)

kor_week_popu_df = pd.DataFrame({'제목' : kor_week_popu_title,
                        '요일': kor_week_popu_week,
                       '코드': kor_week_popu_code})

kor_week_popu_df.to_excel(writer, sheet_name='kor_week_popu_df')

In [7]:
#한국 요일별 조회순
kor_week_watch_title, kor_week_watch_week, kor_week_watch_code =[],[],[]

for i in ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']:
    raw = requests.get('https://comic.naver.com/webtoon/weekdayList?week='+i+'&order=ViewCount&view=image')
    html = BeautifulSoup(raw.text,'html.parser')
    
    #제목
    raw_title_list = html.select('dt')
    for j in raw_title_list[3:] : 
        kor_week_watch_title.append(j.get_text())
    
    container = html.select('ul.img_list li')
    for cont in container:
        #코드
        kor_week_watch_code.append(re.sub(r'[^0-9]', '', str(cont.select_one('dt'))[:45]))
        #요일
        kor_week_watch_week.append(i)

kor_week_watch_df = pd.DataFrame({'제목' : kor_week_watch_title,
                        '요일': kor_week_watch_week,
                       '코드': kor_week_watch_code})

kor_week_watch_df.to_excel(writer, sheet_name='kor_week_watch_df')

In [8]:
#한국 3개 장르별 인기순  (에피소드,옴니버스,스토리)
kor_gen3_popu_title, kor_gen3_popu_genre, kor_gen3_popu_code =[],[],[]


for i in ['episode','omnibus', 'story']:
    raw = requests.get('https://comic.naver.com/webtoon/genre?order=User&view=image&genre='+i)
    html = BeautifulSoup(raw.text,'html.parser')
    
    #제목
    raw_title_list = html.select('dt')
    for j in raw_title_list : 
        kor_gen3_popu_title.append(j.get_text())
    
    container = html.select('ul.img_list li')
    for cont in container:
        #코드
        kor_gen3_popu_code.append(re.sub(r'[^0-9]', '', str(cont.select_one('dt'))[:45]))
        #요일
        kor_gen3_popu_genre.append(i)

kor_gen3_popu_df = pd.DataFrame({'제목' : kor_gen3_popu_title,
                        '장르': kor_gen3_popu_genre,
                       '코드': kor_gen3_popu_code})

kor_gen3_popu_df.to_excel(writer, sheet_name='kor_gen3_popu_df')

In [9]:
#한국 3개 장르별 인기순  (에피소드,옴니버스,스토리)
kor_gen3_watch_title, kor_gen3_watch_genre, kor_gen3_watch_code =[],[],[]


for i in ['episode','omnibus', 'story']:
    raw = requests.get('https://comic.naver.com/webtoon/genre?order=ViewCount&view=image&genre='+i)
    html = BeautifulSoup(raw.text,'html.parser')
    
    #제목
    raw_title_list = html.select('dt')
    for j in raw_title_list : 
        kor_gen3_watch_title.append(j.get_text())
    
    container = html.select('ul.img_list li')
    for cont in container:
        #코드
        kor_gen3_watch_code.append(re.sub(r'[^0-9]', '', str(cont.select_one('dt'))[:45]))
        #요일
        kor_gen3_watch_genre.append(i)

kor_gen3_watch_df = pd.DataFrame({'제목' : kor_gen3_watch_title,
                        '장르': kor_gen3_watch_genre,
                       '코드': kor_gen3_watch_code})

kor_gen3_watch_df.to_excel(writer, sheet_name='kor_gen3_watch_df')

In [10]:
#한국 10개 장르별 인기순  (일상,개그,판타지,액션,드라마,순정,감성,스릴러,시대극,스포츠)
kor_gen10_popu_title, kor_gen10_popu_genre, kor_gen10_popu_code =[],[],[]


for i in ['daily', 'comic', 'fantasy', 'action', 'drama', 'pure', 'sensibility', 'thrill', 'historical', 'sports']:
    raw = requests.get('https://comic.naver.com/webtoon/genre?order=User&view=image&genre='+i)
    html = BeautifulSoup(raw.text,'html.parser')
    
    #제목
    raw_title_list = html.select('dt')
    for j in raw_title_list : 
        kor_gen10_popu_title.append(j.get_text())
    
    container = html.select('ul.img_list li')
    for cont in container:
        #코드
        kor_gen10_popu_code.append(re.sub(r'[^0-9]', '', str(cont.select_one('dt'))[:45]))
        #장르
        kor_gen10_popu_genre.append(i)

kor_gen10_popu_df = pd.DataFrame({'제목' : kor_gen10_popu_title,
                        '장르': kor_gen10_popu_genre,
                       '코드': kor_gen10_popu_code})

kor_gen10_popu_df.to_excel(writer, sheet_name='kor_gen10_popu_df')

In [11]:
#한국 10개 장르별 조회순  (일상,개그,판타지,액션,드라마,순정,감성,스릴러,시대극,스포츠)
kor_gen10_watch_title, kor_gen10_watch_genre, kor_gen10_watch_code =[],[],[]


for i in ['daily', 'comic', 'fantasy', 'action', 'drama', 'pure', 'sensibility', 'thrill', 'historical', 'sports']:
    raw = requests.get('https://comic.naver.com/webtoon/genre?order=ViewCount&view=image&genre='+i)
    html = BeautifulSoup(raw.text,'html.parser')
    
    #제목
    raw_title_list = html.select('dt')
    for j in raw_title_list : 
        kor_gen10_watch_title.append(j.get_text())
    
    container = html.select('ul.img_list li')
    for cont in container:
        #코드
        kor_gen10_watch_code.append(re.sub(r'[^0-9]', '', str(cont.select_one('dt'))[:45]))
        #장르
        kor_gen10_watch_genre.append(i)

kor_gen10_watch_df = pd.DataFrame({'제목' : kor_gen10_watch_title,
                        '장르': kor_gen10_watch_genre,
                       '코드': kor_gen10_watch_code})

kor_gen10_watch_df.to_excel(writer, sheet_name='kor_gen10_watch_df')

In [12]:
writer.save()